In [1]:
import yfinance
import pandas as pd
import json

In [2]:
import sqlite3
connection = sqlite3.connect('data.db')

In [3]:
symbols = []
with open('sp500.json') as file:
    symbols = json.load(file)

len(symbols)

503

In [14]:
for symbol in symbols:
    df: pd.DataFrame = yfinance.download(symbol, start='2004-01-01', end='2024-01-01', interval='1wk')
    df['symbol'] = symbol
    df = df[['symbol', 'Adj Close', 'Volume']].reset_index()
    df.columns = ['date', 'symbol', 'price', 'volume']
    df.to_sql('PRICES', connection, if_exists='append', index=False)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [11]:
df = df[['symbol', 'Adj Close', 'Volume']].reset_index()
df.columns = ['date', 'symbol', 'price', 'volume']
df

,date,symbol,price,volume
0,2019-05-10,AAPL,41.570000,186322500
1,2019-05-13,AAPL,37.099998,79442400
2,2019-05-14,AAPL,39.959999,46661100
3,2019-05-15,AAPL,41.290001,36086100
4,2019-05-16,AAPL,43.000000,38115500
...,...,...,...,...
1164,2023-12-22,AAPL,61.709999,14709400
1165,2023-12-26,AAPL,61.980000,16077200
1166,2023-12-27,AAPL,63.279999,20806000
1167,2023-12-28,AAPL,63.139999,13795600


Скорочена назва	Опис	 Період (неділі)
bb_pr_up	log(ціна / верхня лінія Боллінджера)	13
bb_pr_dn	log(ціна / нижня лінія Боллінджера)	13
rsi	Індекс відносної сили	13
mom	Ціновий імпульс	13 та 26
lr_slope	Кут нахилу лінійної регресії	13
ma	log(ціна / ковзне середнє)	26 та 52
ret	Повернення із застримкою	13 та 26
excess	Надлишкова дохідність із затримкою	13 та 26
ret_div_vol	Дохідність, скоригована на ризик	13
vol	Волатильність доходу	13
beta	Коефіцієнт бета	13 та 26
skew	Асиметричність доходу	13 та 26
corr_rtn_volume	Кореляція між доходом та об’ємами торгівлі	13
corr_rtn_spxrtn	Кореляція між доходом та базовим доходом	13
obv	Обсяг балансу	-
dlr_vol	Обсяг балансу в доларах	-


In [ ]:
import talib

talib.BBANDS(13)
talib.RSI(13)
talib.MOM(13), talib.MOM(26)
talib.LINEARREG_SLOPE(13)
talib.MA(26), talib.MA(52)
talib.BETA(13), talib.BETA(26)

Фундаментал треба десь накачать такий, який можна було б взяти тут:

In [2]:
ticker = yfinance.Ticker('AAPL')
ticker.quarterly_balance_sheet

,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Treasury Shares Number,NaN,0.0,0.0,NaN,NaN
Ordinary Shares Number,15337686000.0,15460223000.0,15550061000.0,15647868000.0,15723406000.0
Share Issued,15337686000.0,15460223000.0,15550061000.0,15647868000.0,15723406000.0
Net Debt,71895000000.0,67280000000.0,81123000000.0,80872000000.0,84928000000.0
Total Debt,104590000000.0,108040000000.0,111088000000.0,109280000000.0,109615000000.0
...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,67150000000.0,73100000000.0,61555000000.0,62482000000.0,55872000000.0
Other Short Term Investments,34455000000.0,32340000000.0,31590000000.0,34074000000.0,31185000000.0
Cash And Cash Equivalents,32695000000.0,40760000000.0,29965000000.0,28408000000.0,24687000000.0
Cash Equivalents,4468000000.0,11218000000.0,1606000000.0,3071000000.0,4637000000.0


In [3]:
ticker.quarterly_cash_flow

,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Free Cash Flow,20694000000.0,37503000000.0,19435000000.0,24287000000.0,25644000000.0
Repurchase Of Capital Stock,-23205000000.0,-20139000000.0,-21003000000.0,-17478000000.0,-19594000000.0
Repayment Of Debt,-3148000000.0,-3984000000.0,0.0,460000000.0,-1996000000.0
Issuance Of Debt,NaN,NaN,0.0,NaN,NaN
Capital Expenditure,-1996000000.0,-2392000000.0,-2163000000.0,-2093000000.0,-2916000000.0
Interest Paid Supplemental Data,NaN,NaN,1213000000.0,717000000.0,1170000000.0
Income Tax Paid Supplemental Data,7276000000.0,7255000000.0,11659000000.0,2126000000.0,4066000000.0
End Cash Position,33921000000.0,41974000000.0,30737000000.0,29898000000.0,27129000000.0
Beginning Cash Position,41974000000.0,30737000000.0,29898000000.0,27129000000.0,21974000000.0
Changes In Cash,-8053000000.0,11237000000.0,839000000.0,2769000000.0,5155000000.0


In [5]:
ticker.quarterly_income_stmt

,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0,0.0
Tax Rate For Calcs,0.158,0.159,0.149715,0.125,0.149
Normalized EBITDA,30736000000.0,43221000000.0,30653000000.0,26783000000.0,31216000000.0
Net Income From Continuing Operation Net Minority Interest,23636000000.0,33916000000.0,22956000000.0,19881000000.0,24160000000.0
Reconciled Depreciation,2836000000.0,2848000000.0,2653000000.0,3052000000.0,2898000000.0
Reconciled Cost Of Revenue,48482000000.0,64720000000.0,49071000000.0,45384000000.0,52860000000.0
EBITDA,30736000000.0,43221000000.0,30653000000.0,26783000000.0,31216000000.0
EBIT,27900000000.0,40373000000.0,28000000000.0,23731000000.0,28318000000.0
Net Interest Income,NaN,NaN,-18000000.0,-18000000.0,-12000000.0
Interest Expense,NaN,NaN,1002000000.0,998000000.0,930000000.0
